In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,593 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,325 kB]
Get:14 http://archive.ubun

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-13 21:25:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.58MB/s    in 0.6s    

2023-04-13 21:25:02 (1.58 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
# Load in the review data from S3 into the dataframe
url = "https://unbearable-1-project-bucket.s3.us-east-2.amazonaws.com/Diabetes_Key_Indicators.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+---+------+------+-----------+------+-----+-------+--------+-------------+-------------+--------+--------+--------+--------+--------+-------+-------+--------+-------+--------------------+--------+--------+--------+--------+--------+-------+-------+--------+--------+-------+--------+-------+------+-------+--------+--------+--------+--------+--------+--------+--------+--------+-------+------+--------+-------+--------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+-------+-----+--------+--------+--------+--------+--------+-------+--------+-------+--------+-------+-------+----+-----+------+--------+--------+--------+--------+--------+-------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+-------+----

In [5]:
new_df = df.select(["DIABETE4",
                    "TOLDHI3",
                    "BPHIGH6",
                    "_BMI5",
                    "SMOKE100",
                    "DRNK3GE5",
                    "EDUCA",
                    "GENHLTH",
                    "_AGEG5YR",
                    "_AGE65YR",
                    "_AGE80",
                    "EXERANY2",
                    "FRUIT2",
                    "VEGETAB2",
                    "INCOME3",
                    "MEDCOST1",
                    "_HISPANC",
                    "_MRACE1",
                    "_PRACE1",
                    "_SEX"])
new_df.show()

+--------+-------+-------+------+--------+--------+-----+-------+--------+--------+------+--------+------+--------+-------+--------+--------+-------+-------+----+
|DIABETE4|TOLDHI3|BPHIGH6| _BMI5|SMOKE100|DRNK3GE5|EDUCA|GENHLTH|_AGEG5YR|_AGE65YR|_AGE80|EXERANY2|FRUIT2|VEGETAB2|INCOME3|MEDCOST1|_HISPANC|_MRACE1|_PRACE1|_SEX|
+--------+-------+-------+------+--------+--------+-----+-------+--------+--------+------+--------+------+--------+-------+--------+--------+-------+-------+----+
|     3.0|    1.0|    3.0|1454.0|     1.0|    null|  4.0|    5.0|    11.0|     2.0|  70.0|     2.0| 101.0|   101.0|    5.0|     2.0|     2.0|    1.0|    1.0| 2.0|
|     1.0|    1.0|    1.0|  null|     2.0|    null|  6.0|    3.0|    10.0|     2.0|  67.0|     1.0| 101.0|   207.0|   77.0|     2.0|     2.0|    2.0|    2.0| 2.0|
|     1.0|    2.0|    1.0|2829.0|     2.0|    null|  4.0|    2.0|    11.0|     2.0|  72.0|     2.0| 101.0|   203.0|    3.0|     2.0|     2.0|    2.0|    2.0| 2.0|
|     1.0|    1.0|    

### Clean up Columns

In [6]:
import pandas as pd
pandas_df = new_df.toPandas()
pandas_df = pandas_df.dropna()

pandas_df

,DIABETE4,TOLDHI3,BPHIGH6,_BMI5,SMOKE100,DRNK3GE5,EDUCA,GENHLTH,_AGEG5YR,_AGE65YR,_AGE80,EXERANY2,FRUIT2,VEGETAB2,INCOME3,MEDCOST1,_HISPANC,_MRACE1,_PRACE1,_SEX
3,1.0,1.0,1.0,3347.0,2.0,1.0,4.0,2.0,9.0,1.0,62.0,1.0,203.0,205.0,7.0,2.0,2.0,1.0,1.0,2.0
9,3.0,2.0,1.0,3994.0,1.0,88.0,4.0,3.0,10.0,2.0,65.0,1.0,101.0,204.0,8.0,2.0,2.0,1.0,1.0,2.0
19,3.0,1.0,1.0,2986.0,2.0,88.0,4.0,2.0,7.0,1.0,53.0,2.0,202.0,207.0,6.0,2.0,2.0,1.0,1.0,2.0
22,3.0,1.0,1.0,3587.0,1.0,88.0,4.0,4.0,10.0,2.0,68.0,2.0,302.0,306.0,8.0,2.0,2.0,1.0,1.0,1.0
28,3.0,1.0,1.0,2905.0,2.0,88.0,5.0,2.0,13.0,2.0,80.0,1.0,330.0,330.0,99.0,2.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438687,1.0,1.0,1.0,2148.0,2.0,8.0,2.0,4.0,10.0,2.0,66.0,1.0,315.0,205.0,3.0,2.0,1.0,2.0,2.0,1.0
438688,2.0,2.0,1.0,2469.0,1.0,88.0,4.0,2.0,3.0,1.0,30.0,1.0,210.0,103.0,5.0,1.0,2.0,99.0,99.0,2.0
438690,3.0,1.0,3.0,3068.0,2.0,88.0,6.0,2.0,7.0,1.0,54.0,1.0,102.0,101.0,10.0,2.0,2.0,2.0,2.0,1.0
438691,3.0,2.0,1.0,2373.0,2.0,88.0,4.0,2.0,10.0,2.0,67.0,1.0,101.0,101.0,6.0,2.0,2.0,99.0,99.0,1.0


In [7]:
# check for unique values to replace for encoding
pandas_df.EDUCA.unique()

array([4., 5., 6., 3., 2., 1., 9.])

In [9]:
pandas_df["EDUCA"] = pandas_df["EDUCA"].replace({4.:"High School Grad"}).replace({6.:"College Grad"}).replace({3.:"Some High School"}).replace({5.:"Some College"}).replace({2.:"Elementary"}).replace({1.:"None"}).replace({9.:""})
pandas_df.EDUCA.unique()


array(['High School Grad', 'Some College', 'College Grad',
       'Some High School', 'Elementary', 'None', ''], dtype=object)

In [62]:
pandas_df.GENHLTH.unique()

array([2., 3., 4., 7., 1., 5., 9.])

In [10]:
pandas_df["GENHLTH"] = pandas_df["GENHLTH"].replace({1.:"Excellent"}).replace({2.:"Very Good"}).replace({3.:"Good"}).replace({4.:"Fair"}).replace({5.:"Poor"}).replace({7.:""}).replace({9.:""})
pandas_df.GENHLTH.unique()

array(['Very Good', 'Good', 'Fair', '', 'Excellent', 'Poor'], dtype=object)

In [11]:
# do replace values for EDUCA, GENHLTH, DRNK3GE5, EXERANY2, MEDCOST1, _SEX, INCOME3, SMOKE100
# If responses are dont know or refused to answer those responses will need to be excluded as they dont tell us anything

pandas_df



,DIABETE4,TOLDHI3,BPHIGH6,_BMI5,SMOKE100,DRNK3GE5,EDUCA,GENHLTH,_AGEG5YR,_AGE65YR,_AGE80,EXERANY2,FRUIT2,VEGETAB2,INCOME3,MEDCOST1,_HISPANC,_MRACE1,_PRACE1,_SEX
3,1.0,1.0,1.0,3347.0,2.0,1.0,High School Grad,Very Good,9.0,1.0,62.0,1.0,203.0,205.0,7.0,2.0,2.0,1.0,1.0,2.0
9,3.0,2.0,1.0,3994.0,1.0,88.0,High School Grad,Good,10.0,2.0,65.0,1.0,101.0,204.0,8.0,2.0,2.0,1.0,1.0,2.0
19,3.0,1.0,1.0,2986.0,2.0,88.0,High School Grad,Very Good,7.0,1.0,53.0,2.0,202.0,207.0,6.0,2.0,2.0,1.0,1.0,2.0
22,3.0,1.0,1.0,3587.0,1.0,88.0,High School Grad,Fair,10.0,2.0,68.0,2.0,302.0,306.0,8.0,2.0,2.0,1.0,1.0,1.0
28,3.0,1.0,1.0,2905.0,2.0,88.0,Some College,Very Good,13.0,2.0,80.0,1.0,330.0,330.0,99.0,2.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438687,1.0,1.0,1.0,2148.0,2.0,8.0,Elementary,Fair,10.0,2.0,66.0,1.0,315.0,205.0,3.0,2.0,1.0,2.0,2.0,1.0
438688,2.0,2.0,1.0,2469.0,1.0,88.0,High School Grad,Very Good,3.0,1.0,30.0,1.0,210.0,103.0,5.0,1.0,2.0,99.0,99.0,2.0
438690,3.0,1.0,3.0,3068.0,2.0,88.0,College Grad,Very Good,7.0,1.0,54.0,1.0,102.0,101.0,10.0,2.0,2.0,2.0,2.0,1.0
438691,3.0,2.0,1.0,2373.0,2.0,88.0,High School Grad,Very Good,10.0,2.0,67.0,1.0,101.0,101.0,6.0,2.0,2.0,99.0,99.0,1.0


In [12]:
pandas_df=pandas_df.mask(pandas_df == "")

In [13]:
pandas_df.isna().sum()

DIABETE4      0
TOLDHI3       0
BPHIGH6       0
_BMI5         0
SMOKE100      0
DRNK3GE5      0
EDUCA       269
GENHLTH     208
_AGEG5YR      0
_AGE65YR      0
_AGE80        0
EXERANY2      0
FRUIT2        0
VEGETAB2      0
INCOME3       0
MEDCOST1      0
_HISPANC      0
_MRACE1       0
_PRACE1       0
_SEX          0
dtype: int64